# Data wrangling

The purpose of this notebook is to make our hands durty (i.e to handle with the datasets). We will try to to clean and understand what is in the data

In [2]:
# some basic imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
 

In [226]:
#load data
raw_data_folder = '../../data/raw_data/' 
characters_header_name = ['movie_wiki_id','freebase_movie_id','movie_release_date', 'character_name', 'birth', 'gender', 'height', 'ethnicity', 'name', 'release_age', 'freebase_character_map_id', 'freebase_character_id', 'freebase_actor_id']
movies_header_name = ['wiki_id', 'freebase_id', 'name', 'release_date', 'revenue', 'runtime', 'languages', 'country', 'genres']


characters_df = pd.read_csv( raw_data_folder + 'character.metadata.tsv', sep = '\t', names = characters_header_name)
movies_df = pd.read_csv( raw_data_folder + 'movie.metadata.tsv', sep = '\t', names= movies_header_name, index_col= 'wiki_id' )

characters_df.head(5)

,movie_wiki_id,freebase_movie_id,movie_release_date,character_name,birth,gender,height,ethnicity,name,release_age,freebase_character_map_id,freebase_character_id,freebase_actor_id
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg


In [227]:
movies_df.head()

,freebase_id,name,release_date,revenue,runtime,languages,country,genres
wiki_id,,,,,,,,
975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


In [228]:
#check if all movies has an wiki_id
movies_df[movies_df.index.isna()] 

,freebase_id,name,release_date,revenue,runtime,languages,country,genres
wiki_id,,,,,,,,


In [229]:
#clean movies dataframe 

#remove useless columns
movies_clean_df = movies_df.copy()
movies_clean_df['freebase_id']
movies_clean_df.drop('freebase_id', axis= 1, inplace= True)

#remove all freebase code such as "/m/02h40lc"
movies_clean_df['languages'] = movies_clean_df['languages'].str.replace('"/m/\w*": ', '', regex= True)
movies_clean_df['country'] = movies_clean_df['country'].str.replace('"/m/\w*": ', '', regex= True)
movies_clean_df['genres'] = movies_clean_df['genres'].str.replace('"/m/\w*": ', '', regex= True)

#remove ""
movies_clean_df['languages'] = movies_clean_df['languages'].str.replace('"', '', regex= False)
movies_clean_df['country'] = movies_clean_df['country'].str.replace('"', '', regex= False)
movies_clean_df['genres'] = movies_clean_df['genres'].str.replace('"', '', regex= False)


#remove {}
movies_clean_df['languages'] = movies_clean_df['languages'].str.replace('{', '', regex= False)
movies_clean_df['languages'] = movies_clean_df['languages'].str.replace('}', '', regex= False)

movies_clean_df['country'] = movies_clean_df['country'].str.replace('{', '', regex= False)
movies_clean_df['country'] = movies_clean_df['country'].str.replace('}', '', regex= False)

movies_clean_df['genres'] = movies_clean_df['genres'].str.replace('{', '', regex= False)
movies_clean_df['genres'] = movies_clean_df['genres'].str.replace('}', '', regex= False)


#remove Language in languages columns 
movies_clean_df['languages'] = movies_clean_df['languages'].str.replace('Language', '', regex= False)
movies_clean_df['country'] = movies_clean_df['languages'].str.replace('Language', '', regex= False)
movies_clean_df['genres'] = movies_clean_df['genres'].str.replace('Language', '', regex= False)





In [230]:
movies_clean_df.head()

,name,release_date,revenue,runtime,languages,country,genres
wiki_id,,,,,,,
975900,Ghosts of Mars,2001-08-24,14010832.0,98.0,English,English,"Thriller, Science Fiction, Horror, Adventure, ..."
3196793,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,English,English,"Mystery, Biographical film, Drama, Crime Drama"
28463795,Brun bitter,1988,NaN,83.0,Norwegian,Norwegian,"Crime Fiction, Drama"
9363483,White Of The Eye,1987,NaN,110.0,English,English,"Thriller, Erotic thriller, Psychological thriller"
261236,A Woman in Flames,1983,NaN,106.0,German,German,Drama


In [238]:
#clean characters dataframe
characters_clean_df = characters_df.copy()

#remove useless columns
characters_clean_df.drop('freebase_movie_id', axis= 1, inplace= True)
characters_clean_df.drop('movie_release_date', axis= 1, inplace= True)
characters_clean_df.drop('freebase_character_map_id', axis= 1, inplace= True)
characters_clean_df.drop('freebase_character_id', axis= 1, inplace= True)

In [244]:
#maybe drop ethinity columns due to proportion of avaible data
characters_clean_df['ethnicity'].isna().sum()/characters_clean_df.shape[0] * 100

76.46654196317031

In [245]:
characters_clean_df.head()


,movie_wiki_id,character_name,birth,gender,height,ethnicity,name,release_age,freebase_actor_id
0,975900,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/03wcfv7
1,975900,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0346l4
2,975900,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/01vw26l
3,975900,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/034hyc
4,975900,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/01y9xg
